# Dispositivos de captura

Para hacer experimentos de visión artificial nos interesa acceder cómodamente a cualquier fuente de imágenes, almacenada localmente o disponible en un servidor remoto. Por ejemplo:

- archivos de imagen (jpg, png, tif, bmp, etc.) 

- archivos de video (avi, mp4, etc.)

- secuencias "en vivo" tomadas de una webcam, cámara ip, o smartphone.

El paquete `umucv` simplifica esta tarea.

## autoStream

La función `autoStream()` proporciona una secuencia de imágenes seleccionada en la línea de comandos. El dispositivo de captura se indica con `--dev`, la resolución deseada  con `--size`, forzar un reescalado con `--resize`, la repetición de un video con `--loop` y la ejecución frame a frame con `--step`.

    from umucv.stream import autoStream

    source = autoStream()

    for (key, frame) in source:
       
        #procesamos cada fotograma (dependiendo de tecla pulsada)

Esta función gestiona automáticamente la tecla ESC para terminar, SPACE para hacer pausas, '.' para activar/desactivar el avance frame a frame (pulsando otra tecla no asignada), y S para guardar en disco la imagen de entrada actual.

Se puede pedir ayuda con `--dev=help`

Por omisión el dispositivo es la cámara /dev/video0. Para cambiar de webcam y solicitar un tamaño hacemos:

    ./programa.py --dev=1 --size=320x240

Podemos procesar una lista de imágenes tomadas de un directorio:

    ./programa.py --dev=glob:../images/ccorr/scenes/*.png

Otra forma de hacer lo mismo, pero visualizando las imágenes disponibles en una ventana, avanzando de una en una con las teclas del ratón:

    ./programa.py --dev=dir:../images/ccorr/scenes/*.png

Se admiten archivos de vídeo (locales o remotos):

    ./programa.py --dev=../images/rot4.mjpg
        
También se admiten *streams* en vivo remotos, generados por [cámaras online](https://en.wikipedia.org/wiki/IP_camera#Video_standards). Por ejemplo:

    ./programa.py --dev=https://iihrwc02.iowa.uiowa.edu/axis-cgi/mjpg/video.cgi?resolution=704x480

Algunos canales de TV emiten streams. Por ejemplo, aquí está la TVE1:

    http://hlsliveamdgl7-lh.akamaihd.net/i/hlsdvrlive_1@583042/index_0400_av-p.m3u8?sd=10&rebase=on


La *app* "IP Webcam" incluye un servidor mjpeg en los smartphones, que permite utilizarlos como fuente de imágenes para los ejercicios.

    ./programa.py --dev=http://155.54.X.Y:8080/video

La *app* "VXG RTSP Server (IP camera)" es un servidor rtsp con codificación H264, que utiza menos ancho de banda:

    ./programa.py --dev=rtsp://155.54.X.Y:5540/ch0
    
En los teléfonos se recomienda desactivar el audio, poner un tamaño de imagen no muy grande y en su caso limitar el bitrate.

Para leer vídeos de youtube se puede utilizar la utilidad [`code/youtube.py`](../code/youtube.py). Ejecútalo sin parámetros para ver la ayuda. Por ejemplo:

    ./programa.py --dev=`./youtube.py 6 C6LuDdY-KrQ`

Se pueden definir abreviaturas en un archivo **alias.txt**.

## Servidores

Hemos incluido dos utilidades para crear "streams" en formato mjpeg:

- ./`mjpegserver.py`: crea un stream mjpeg a partir cualquier dispositivo de los anteriores. Por ejemplo:

        ./mjpegserver.py --dev=picam --size=320x240 --quality=50
        
    Este stream se captura con
    
        ./programa.py --dev=http://<IP-DEL-SERVIDOR>:8087/cam.mjpg
        
- ./`vlcmjpeg.sh`: es un script de bash para llamar a `vlc` con cualquier fuente de video y generar un stream mjpeg:

        ./vlcmjpeg.sh https://www.youtube.com/watch?v=aBr2kKAHN6M
        
    (Es conveniente abrir el vídeo primero con vlc normal para comprobar que se lee bien, y en su caso aceptar certificados.)
    
    Este stream se captura con
    
        ./programa.py --dev=http://<IPDELSERVIDOR>:8090
    
    Podemos transmitir la pantalla de nuestro ordenador:
        
        ./vlcmjpeg.sh 'screen:// :screen-fps=10 :screen-width=700 :screen-height=500 :screen-top=300'
        
    O la webcam:
    
        ./vlcmjpeg.sh 'v4l2://'    
    
    En, general, podemos emitir cualquier fuente de video admitida por `vlc` y capturarla con `autoStream()`.

### Visualización en páginas web

Los documentos html permiten visualizar video directamente con elementos `<video>`. Para visualizar streams `mjpg` en vivo se usa directamente un elemento `<img src='http:...' >`. 

## Control de la webcam

En la consola de linux usamos **v4l2-ctl**. Los controles disponibles en cada cámara se consultan con:

    v4l2-ctl -l

Activar o desactivar la eliminación de oscilaciones producidas por la luz eléctrica:

        v4l2-ctl -d /dev/video0 -c power_line_frequency=1
        v4l2-ctl -d /dev/video0 -c power_line_frequency=0

Fijar el nivel de exposición:

    v4l2-ctl -d /dev/video0 -c exposure_auto=1 -c exposure_absolute=100
    v4l2-ctl -d /dev/video0 -c exposure_auto=1 -c exposure_absolute=1000

Exposición automática:

    v4l2-ctl -d /dev/video0 -c exposure_auto=3

Enfoque fijo:

    v4l2-ctl -d /dev/video0 -c focus=255
    v4l2-ctl -d /dev/video0 -c focus=0

La aplicación *guvcview* permite modificar los parámetros con un interfaz gráfico. Si la invocamos con la opción --control-panel se puede usar a la vez que nuestra aplicación.

El reproductor multimedia VLC (disponible en Linux y Windows) también lo permite.